In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-02 20:45:51.932450: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 20:45:51.932528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 20:45:51.936124: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-02 20:45:51.956108: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-02 20:45:54.608682: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../cell_images/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/27558
[INFO]: Processed 2000/27558
[INFO]: Processed 3000/27558
[INFO]: Processed 4000/27558
[INFO]: Processed 5000/27558
[INFO]: Processed 6000/27558
[INFO]: Processed 7000/27558
[INFO]: Processed 8000/27558
[INFO]: Processed 9000/27558
[INFO]: Processed 10000/27558
[INFO]: Processed 11000/27558
[INFO]: Processed 12000/27558
[INFO]: Processed 13000/27558
[INFO]: Processed 14000/27558
[INFO]: Processed 15000/27558
[INFO]: Processed 16000/27558
[INFO]: Processed 17000/27558
[INFO]: Processed 18000/27558
[INFO]: Processed 19000/27558
[INFO]: Processed 20000/27558
[INFO]: Processed 21000/27558
[INFO]: Processed 22000/27558
[INFO]: Processed 23000/27558
[INFO]: Processed 24000/27558
[INFO]: Processed 25000/27558
[INFO]: Processed 26000/27558
[INFO]: Processed 27000/27558
(27558, 224, 224, 3)
(27558,)


In [8]:
from tensorflow.keras.models import Model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
model = Model(inputs=base_model.input, outputs=base_model.output)
# Extract features
features = model.predict(data, batch_size=32, verbose=1)
 
# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-02 20:46:56.467456: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 36183 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0
2024-04-02 20:47:29.563881: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-02 20:47:31.187537: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-02 20:47:31.409602: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


862/862 [==============================] - 30s 22ms/step
(27558, 62720)


In [9]:
data = features_flatten

In [10]:
import gc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
#tf.config.experimental.enable_tensor_float_32_execution(True)
#tf.config.threading.set_inter_op_parallelism_threads(1)
#tf.config.threading.set_intra_op_parallelism_threads(1)
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16":
        labels = to_categorical(labels)

    elif name == "VGG19":  
        labels = to_categorical(labels)
    else:
        data = data
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "VGG19":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [11]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9524673439767779
Sensitivity: 0.9564586357039188
Specificity: 0.9484760522496372
AUC-ROC: 0.952467343976778
MCC: 0.9049635212921171
Precision: 0.9488840892728582
F1 Score: 0.9526563064691002

[INFO] Fold 2 / 10 for LR
Accuracy: 0.9528301886792453
Sensitivity: 0.9528301886792453
Specificity: 0.9528301886792453
AUC-ROC: 0.9528301886792453
MCC: 0.9056603773584906
Precision: 0.9528301886792453
F1 Score: 0.9528301886792453

[INFO] Fold 3 / 10 for LR
Accuracy: 0.9524673439767779
Sensitivity: 0.9499274310595065
Specificity: 0.9550072568940493
AUC-ROC: 0.9524673439767779
MCC: 0.904946363932156
Precision: 0.9547775346462436
F1 Score: 0.9523463077482721

[INFO] Fold 4 / 10 for LR
Accuracy: 0.9550072568940493
Sensitivity: 0.9506531204644412
Specificity: 0.9593613933236574
AUC-ROC: 0.9550072568940492
MCC: 0.9100490207783999
Precision: 0.95900439238653
F1 Score: 0.9548104956268221

[INFO] Fold 5 / 10 for LR
Accuracy: 0.9524673439767779
Sensitivity: 0.95355587808418
Specificity: 0.951378

In [12]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.9056603773584906
Sensitivity: 0.8780841799709724
Specificity: 0.9332365747460087
AUC-ROC: 0.9056603773584906
MCC: 0.8125575092107895
Precision: 0.9293394777265745
F1 Score: 0.9029850746268656

[INFO] Fold 2 / 10 for NB
Accuracy: 0.908200290275762
Sensitivity: 0.8831640058055152
Specificity: 0.9332365747460087
AUC-ROC: 0.9082002902757621
MCC: 0.8174259742947356
Precision: 0.9297173414820473
F1 Score: 0.9058429475251208

[INFO] Fold 3 / 10 for NB
Accuracy: 0.918722786647315
Sensitivity: 0.8969521044992743
Specificity: 0.9404934687953556
AUC-ROC: 0.9187227866473149
MCC: 0.8382405395908111
Precision: 0.9377845220030349
F1 Score: 0.9169139465875372

[INFO] Fold 4 / 10 for NB
Accuracy: 0.9074746008708273
Sensitivity: 0.872278664731495
Specificity: 0.9426705370101597
AUC-ROC: 0.9074746008708272
MCC: 0.816975779189162
Precision: 0.9383294301327089
F1 Score: 0.9040992854456563

[INFO] Fold 5 / 10 for NB
Accuracy: 0.9045718432510885
Sensitivity: 0.862119013

In [13]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.8991291727140783
Sensitivity: 0.9847605224963716
Specificity: 0.8134978229317852
AUC-ROC: 0.8991291727140784
MCC: 0.8102291575785842
Precision: 0.8407682775712515
F1 Score: 0.9070855614973262

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.8929608127721336
Sensitivity: 0.978955007256894
Specificity: 0.806966618287373
AUC-ROC: 0.8929608127721336
MCC: 0.7978098080159574
Precision: 0.8352941176470589
F1 Score: 0.9014366855997327

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.9107402031930334
Sensitivity: 0.9833091436865021
Specificity: 0.8381712626995645
AUC-ROC: 0.9107402031930333
MCC: 0.8302717899312824
Precision: 0.858681875792142
F1 Score: 0.9167794316644113

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.9071117561683599
Sensitivity: 0.9796806966618288
Specificity: 0.8345428156748912
AUC-ROC: 0.9071117561683599
MCC: 0.8229372334831262
Precision: 0.8555133079847909
F1 Score: 0.9133964817320704

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.9038461538461539
Sensitivity: 0.970

In [14]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.9256168359941945
Sensitivity: 0.888243831640058
Specificity: 0.9629898403483309
AUC-ROC: 0.9256168359941944
MCC: 0.8536215892834187
Precision: 0.96
F1 Score: 0.9227289860535243

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.9296081277213353
Sensitivity: 0.8925979680696662
Specificity: 0.9666182873730044
AUC-ROC: 0.9296081277213353
MCC: 0.861579798519151
Precision: 0.9639498432601881
F1 Score: 0.9269027882441597

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.9285195936139332
Sensitivity: 0.9020319303338171
Specificity: 0.9550072568940493
AUC-ROC: 0.9285195936139332
MCC: 0.8582443154181518
Precision: 0.9524904214559387
F1 Score: 0.9265747297800969

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.9267053701015965
Sensitivity: 0.8838896952104499
Specificity: 0.969521044992743
AUC-ROC: 0.9267053701015965
MCC: 0.85655696808338
Precision: 0.9666666666666667
F1 Score: 0.9234268385140257

[INFO] Fold 5 / 10 for Random Forest
Accuracy: 0.9

In [15]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 12401, number of negative: 12401
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 33.919744 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15993600
[LightGBM] [Info] Number of data points in the train set: 24802, number of used features: 62720
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Accuracy: 0.9506531204644412
Sensitivity: 0.944121915820029
Specificity: 0.9571843251088534
AUC-ROC: 0.9506531204644413
MCC: 0.901383144151396
Precision: 0.9566176470588236
F1 Score: 0.9503287070854639

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Number of positive: 12401, number of negative: 12401
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 34.776795 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15993600
[LightGBM] [Info] Num

In [16]:
# Save results to a CSV file
results_df.to_csv('DS1_Efficient.csv', index=False)

In [17]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.952391,0.952754,0.952028,0.952391,0.904817,0.952095,0.952407,2657.859375,5672.050638
1,NB,0.907904,0.875536,0.940271,0.907904,0.817590,0.936160,0.904786,6594.167969,348.413208
2,KNN,0.902533,0.979461,0.825604,0.902533,0.814849,0.848975,0.909522,6564.792969,917.285128
3,Random Forest,0.924849,0.886857,0.962841,0.924849,0.852227,0.959832,0.921860,6627.398438,4630.692169
4,LGBM,0.949452,0.944118,0.954786,0.949452,0.899014,0.954355,0.949178,37747.328125,10413.698278
